# 1 - Dependencies

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
#mount driver , and unzip , read the pickeled data
from google.colab import drive
drive.mount('/content/gdrive')
!pip install transformers[torch]
# !pip install accelerate -U
! pip install -U accelerate
! pip install -U transformers

In [ ]:
!pip install transformers
# !pip install transformers==4.28.0
!pip install sentencepiece
import nltk
nltk.download('averaged_perceptron_tagger')
import transformers
from transformers import TFBertTokenizer
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification ,BertModel
!pip install tqdm
from tqdm import tqdm  # Import tqdm
from transformers import BertConfig
!pip install simpletransformers
from simpletransformers.ner import NERModel,NERArgs

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


# 1 - Fine tune BERT Model for text classififcation ✅✅✅ DONE


In [1]:
# training on transcript-middle

Final_labled_dat = pd.read_csv("/content/gdrive/MyDrive/3owais_Capital/Earning_call/FINAL_DATASET_PROCESSED_SUMMARIZED_NER_LABELED.csv")
data = pd.DataFrame(Final_labled_dat)

#split the data
data = data[['preprocessed_transcript' , 'stock_price']]
data=data[:800]
# data.head()

data['stock_price_state'] = data['stock_price'].apply(lambda x: 1 if x == 'increase' else 0) #encode stock price


"""i will take the 512 length availble from every text from the middle of the ranscript
, it has more info than the beginning of the transcript beginning , might not be a good practice but due to computing limitations i can go further"""


# Extract middle portion of the transcript , 512 tokens that bert allow , as i noticed , it about after 1000~2000 wors the call starts talking numbers , whicا is more importnant the beginning
middle_start = 2500
middle_end = 3012
data['transcript_middle'] = data['preprocessed_transcript'].apply(lambda x: x[middle_start:middle_end])
data=data[['transcript_middle' ,'stock_price_state' ]]
# data=data.drop(columns=['Unnamed: 0'])
data.head(300)

NameError: ignored

In [ ]:


data['transcript_middle'][299]

' on the stellar year of execution in  second our progress in making avacopan a pipelineinadrug and third how we are pushing the frontiers of our science to broaden our scope into additional very high value areas with highly innovative approaches in for example oncologyill start by reviewing progress on the goals we set at the start of  as shown on slide  the achievements of chemocentryx in  were considerable our nda for avacopan was accepted for review with a pdufa date in july of this year we have made maj'

In [ ]:
#Split
X = list(data['transcript_middle'])
y= list(data['stock_price_state'])

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2,stratify=y)
X_train

#load bert tokenizer
Classifier_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
Classifier_model = BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=2)


X_train_tokenized = Classifier_tokenizer(X_train, max_length=512, truncation=True, padding='max_length')
X_val_tokenized = Classifier_tokenizer(X_val, max_length=512, truncation=True, padding='max_length')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
len(X_train_tokenized['input_ids'])

In [ ]:
#  Create torch dataset class for labels and data , data must be a dataset object

class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

#convert data into data object from pytorch.Dataset
train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)


def compute_metrics(p):
    print(type(p))
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}


In [ ]:
# Define Trainer
args = TrainingArguments(
    output_dir="/content/gdrive/MyDrive/3owais_Capital/Final_TextClassifier_2_800_500wrmstps/results",    # output directory
    num_train_epochs=2,
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=16,  # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                # strength of weight decay
    logging_dir='/content/gdrive/MyDrive/3owais_Capital/Final_TextClassifier_2_800_500wrmstps/logs',
    logging_steps=10,
    evaluation_strategy = "epoch"
    )

trainer = Trainer(
    model=Classifier_model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics)


In [ ]:

trainer.train()


In [ ]:

trainer.save_model('/content/gdrive/MyDrive/3owais_Capital/TextClassification_3_400_350wrmstps/Fine_tuned_Bert')
# Configuration saved in CustomModel/config.json
# Model weights saved in CustomModel/pytorch_model.bin

In [ ]:
trainer.evaluate()

In [ ]:
#get and use model
model_2 = BertForSequenceClassification.from_pretrained("/content/gdrive/MyDrive/3owais_Capital/TextClassification_3_400_350wrmstps/Fine_tuned_Bert")
model_2.to('cpu')
Classifier_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


text = "We recognize that the 2022 revenue guidance that we set forth today is much lower than consensus estimates. We have reflected the estimated negative impacts of the lower loan origination environment, along with the knock-on effects of the volume-related slowdowns in new product adoption into our 2022 guidance.As a result, we believe that this guidance range represents an achievable baseline revenue expectation for Blend in 2022 that forecasts consistent strong growth in the Blend platform"

inputs = Classifier_tokenizer(text,padding = True, truncation = True, return_tensors='pt')
outputs = model_2(**inputs)

predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
predictions = predictions.cpu().detach().numpy()
print(predictions)

if predictions[0][0] >0.5 :
  print("The stock price is decreasing")
else:
  print("The stock price is increasing")

[[0.40388164 0.59611833]]
The stock price is increasing


# 2 - Construct sentiment analysis model based on Bert

In [ ]:
Final_labled_dat = pd.read_csv("/content/gdrive/MyDrive/3owais_Capital/Earning_call/FINAL_DATASET_PROCESSED_SUMMARIZED_NER_LABELED.csv")
dataa = pd.DataFrame(Final_labled_dat)

#split the data
SA_data = dataa[['preprocessed_transcript' , 'sentiment']]
SA_data=SA_data[:800]

#encode sentiment
SA_data['sentiment_state'] = SA_data['sentiment'].apply(lambda x: 1 if x == 'positive' else 0)
#get the middle
middle_start = 2500
middle_end = 3012
SA_data['transcript_middle'] = SA_data['preprocessed_transcript'].apply(lambda x: x[middle_start:middle_end])


SA_data=SA_data[['transcript_middle' ,'sentiment_state' ]]
dataa['transcript'][0]

In [ ]:
#Split
X = list(SA_data['transcript_middle'])
y= list(SA_data['sentiment_state'])

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2,stratify=y)
X_train

SA_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
SA_model = BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=2)


# SA_tokenizer = BertTokenizer.from_pretrained('ProsusAI/finbert') ❌❌yielded bad results
# SA_model = BertForSequenceClassification.from_pretrained('ProsusAI/finbert',num_labels=2,ignore_mismatched_sizes=True)


X_train_tokenized = SA_tokenizer(X_train, max_length=512, truncation=True, padding='max_length')
X_val_tokenized = SA_tokenizer(X_val, max_length=512, truncation=True, padding='max_length')

In [ ]:
len(X_train_tokenized['input_ids'])

640

In [ ]:
# Create torch dataset class for labels and data , data must be a dataset object
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

#convert data into data object from pytorch.Dataset
train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)


def compute_metrics(p):
    print(type(p))
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [ ]:
#define trainer

args = TrainingArguments(
    output_dir="/content/gdrive/MyDrive/3owais_Capital/Senti_2_800_bertUncased/results",    # output directory
    num_train_epochs=2,
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=16,  # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,                # strength of weight decay
    learning_rate=2e-5,
    logging_dir='/content/gdrive/MyDrive/3owais_Capital/Senti_2_800_bertUncased/logs',
    logging_steps=10,
    evaluation_strategy = "epoch"
    )

trainer = Trainer(
    model=SA_model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics)


trainer.train()



In [ ]:
trainer.save_model('/content/gdrive/MyDrive/3owais_Capital/Senti_2_800_bertUncased/Fine_tuned_Bert')
# Configuration saved in CustomModel/config.json
# Model weights saved in CustomModel/pytorch_model.bin

In [ ]:
trainer.evaluate()

In [ ]:
#get and use model
saved_model = BertForSequenceClassification.from_pretrained\
    ("/content/gdrive/MyDrive/3owais_Capital/Senti_2_800_bertUncased/Fine_tuned_Bert")
saved_model.to('cpu')
SA_tokenizer = BertTokenizer.from_pretrained('ProsusAI/finbert')

text = "We recognize that the 2022 revenue guidance that we set forth today is much lower than consensus estimates. We have reflected the estimated negative impacts of the lower loan origination environment, along with the knock-on effects of the volume-related slowdowns in new product adoption into our 2022 guidance.As a result, we believe that this guidance range represents an achievable baseline revenue expectation for Blend in 2022 that forecasts consistent strong growth in the Blend platform"

inputs = SA_tokenizer(text,padding = True, truncation = True, return_tensors='pt')
outputs = saved_model(**inputs)

predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
predictions = predictions.cpu().detach().numpy()
print(predictions)

if predictions[0][0] >0.5:
  print("Negative")
else:
  print("Positive")


[[0.79214585 0.20785421]]
Negative


# 3 - NER Code , fine tuning.

In [ ]:
ner_data = pd.read_csv("/content/gdrive/MyDrive/3owais_Capital/NER_DATA/NER_LABELED_DATA.csv")
ner_data = pd.DataFrame(ner_data)
ner_data.head()

,Sentence,Words,Entities
0,"industry perspective, we have revised upward o...",industry,O
1,"industry perspective, we have revised upward o...","perspective,",O
2,"industry perspective, we have revised upward o...",we,O
3,"industry perspective, we have revised upward o...",have,O
4,"industry perspective, we have revised upward o...",revised,O


In [ ]:
df = pd.DataFrame(ner_data)

#This function adds sentence_id instead of pushing the full sentence to the model

def process_sentences(df):
    new_sentences = []
    current_sentence = df['Sentence'][0]
    sentence_counter = 1

    for sentence in df['Sentence']:
        if sentence == current_sentence:
            new_sentences.append(sentence_counter)
        else:
            sentence_counter += 1
            current_sentence = sentence
            new_sentences.append(sentence_counter)

    # new_sentences.append("Sentence 2")

    df['Sentence_id'] = new_sentences
    return df

processed_df = process_sentences(df)
# print(processed_df)

processed_df.head(5)

,Sentence,Words,Entities,Sentence_id
0,"industry perspective, we have revised upward o...",industry,O,1
1,"industry perspective, we have revised upward o...","perspective,",O,1
2,"industry perspective, we have revised upward o...",we,O,1
3,"industry perspective, we have revised upward o...",have,O,1
4,"industry perspective, we have revised upward o...",revised,O,1


In [ ]:
X = processed_df[['Sentence_id', 'Words']]
y = processed_df['Entities']

x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.3)

# Building up train data and test data
train_data = pd.DataFrame({"Sentence_id": x_train["Sentence_id"], "Words": x_train["Words"], "Entities": y_train})
val_data = pd.DataFrame({"Sentence_id": x_val["Sentence_id"], "Words": x_val["Words"], "Entities": y_val})


train_data.rename(columns={'Sentence_id': 'sentence_id', 'Words': 'words' , 'Entities':'labels'}, inplace=True)
val_data.rename(columns={'Sentence_id': 'sentence_id', 'Words': 'words' , 'Entities':'labels'}, inplace=True)



In [ ]:
labels = processed_df['Entities'].unique().tolist()
labels

In [ ]:
args  = NERArgs(
output_dir="/content/gdrive/MyDrive/3owais_Capital/NER_Model_10epochs/results",
num_train_epochs = 10,
learning_rate = 1e-4,
overwrite_output_dir =True,
train_batch_size = 32,
eval_batch_size = 32
)

model = NERModel('bert' , 'bert-base-cased', labels = labels , args= args, use_cuda=False)


In [ ]:
#train
model.train_model(train_data, eval_data = val_data,acc=accuracy_score)

/usr/local/lib/python3.10/dist-packages/simpletransformers/ner/ner_utils.py:190: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  return [


  0%|          | 0/1 [00:00<?, ?it/s]

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/4 [00:00<?, ?it/s]

Running Epoch 1 of 10:   0%|          | 0/4 [00:00<?, ?it/s]

...

Running Epoch 6 of 10:   0%|          | 0/4 [00:00<?, ?it/s]

Running Epoch 7 of 10:   0%|          | 0/4 [00:00<?, ?it/s]

Running Epoch 8 of 10:   0%|          | 0/4 [00:00<?, ?it/s]

Running Epoch 9 of 10:   0%|          | 0/4 [00:00<?, ?it/s]

(40, 0.6484134271740913)

In [ ]:
#save
model.save_model("/content/gdrive/MyDrive/3owais_Capital/NER_Model_10epochs/results")


In [ ]:
from transformers import BertTokenizer
import re


ner_model_path = "/content/gdrive/MyDrive/3owais_Capital/NER_Model_10epochs/results"
modelll = NERModel('bert',ner_model_path, use_cuda=False)


tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
text = " revenue grew $3 million dollar year-over-year"
tokens = tokenizer.tokenize(tokenizer.decode(tokenizer.encode(text)))
prediction, model_output = modelll.predict(tokens)
prediction

In [ ]:
# result, model_outputs, preds_list = model.eval_model(val_data)


In [ ]:
# result

#FAILED TRIALS ❌❌

## training on 'summary' column ❌❌yielded terrible results

In [ ]:
Final_labled_dat = pd.read_csv("/content/gdrive/MyDrive/3owais_Capital/Earning_call/Final_labled_data.csv")
dataa = pd.DataFrame(Final_labled_dat)

#split the data
data = dataa[['summary' , 'stock_price']]
data=data[:400]
# data.head()

data['stock_price_state'] = data['stock_price'].apply(lambda x: 1 if x == 'increase' else 0) #encode stock price


data=data[['summary' ,'stock_price_state' ]]
data.head(300)

## PegasusModel from google for sentiment analyis ,❌❌yielded terrible results

In [ ]:
# !pip install torch==1.8.2+cu111 torchvision==0.9.2+cu111 torchaudio===0.8.2 -f https://download.pytorch.org/whl/lts/1.8/torch_lts.html
# # Importing dependencies from transformers
# from transformers import PegasusForConditionalGeneration, PegasusTokenizer
# # Load tokenizer
# tokenizer = PegasusTokenizer.from_pretrained("google/pegasus-xsum")
# # Load model
# model = PegasusForConditionalGeneration.from_pretrained("google/pegasus-xsum")

In [ ]:
# # Iterate through each row
# for index, row in train_df[:1].iterrows():
#     text = row["transcript"]

#     # Create tokens - number representation of our text
#     tokens = tokenizer(text, truncation=True, padding="longest", return_tensors="pt")

#     summary_ids  = model.generate(**tokens,max_length=512)
#     print(summary_ids)
#     summary = tokenizer.decode(summary_ids[0])
#     # Append the summary to the "summary" column
#     train_df.at[index, "summary"] = summary

# # Save the DataFrame with summaries to a new CSV file
# train_df.to_csv('/content/gdrive/MyDrive/3owais/Earning_call/summarized_data.csv', index=False)

# csv_data = pd.read_csv("/content/gdrive/MyDrive/3owais/Earning_call/summarized_data.csv")
# summary_csv = pd.DataFrame(csv_data)

# summary_csv.head(2)
# summary_csv['summary'][0]

In [ ]:
# import nltk
# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('wordnet')
# import transformers
# from transformers import TFBertTokenizer
# from nltk.corpus import stopwords
# from nltk.stem import WordNetLemmatizer
# from sklearn.model_selection import train_test_split

# def preprocess_and_tokenize(texts, labels, tokenizer):
#     # Instantiate lemmatizer and stopwords
#     lemmatizer = WordNetLemmatizer()
#     stop_words = set(stopwords.words('english'))

#     tokenized_texts = []
#     for text in texts:
#         # Tokenization
#         tokens = tokenizer.tokenize(text)

#         # Removing special characters and non-alphabetic tokens
#         tokens = [token for token in tokens if token.isalpha()]

#         # Removing stopwords and lemmatization
#         tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words]

        # Joining tokens back into a sentence
    #     processed_text = " ".join(tokens)

    #     # Adding processed text to the list
    #     tokenized_texts.append(processed_text)

    # # Encoding tokenized texts
    # encoding = tokenizer.batch_encode_plus(
    #     tokenized_texts,
    #     padding='max_length',
    #     truncation=True,
    #     return_tensors='tf',
    #     add_special_tokens=True
    # )

    # input_ids = encoding['input_ids']
    # token_type_ids = encoding['token_type_ids']
    # attention_mask = encoding['attention_mask']

    # return {'input_ids': input_ids, 'token_type_ids': token_type_ids, 'attention_mask': attention_mask, 'labels': labels}





## 1-FineTune Bert , layers finue tuned

In [ ]:
import os
import re
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
!pip install transformers==2.8.0
from transformers import BertTokenizer
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
# %%time
import torch
import torch.nn as nn
from transformers import BertModel

if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

  Using cached transformers-2.8.0-py3-none-any.whl (563 kB)
  Using cached tokenizers-0.5.2.tar.gz (64 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached boto3-1.28.32-py3-none-any.whl (135 kB)
  Using cached sacremoses-0.0.53-py3-none-any.whl
  Using cached botocore-1.31.32-py3-none-any.whl (11.1 MB)
  Using cached jmespath-1.0.1-py3-none-any.whl (20 kB)
  Using cached s3transfer-0.6.2-py3-none-any.whl (79 kB)
  Using cached urllib3-1.26.16-py2.py3-none-any.whl (143 kB)
  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for tokenizers
Failed to build tokenizers
ERROR: Could not build wheels for tokenizers, which is required to install pyproject.toml-based

In [ ]:
########################################################################################
#  Load train data
########################################################################################

data =  pd.read_csv("/content/gdrive/MyDrive/3owais_Capital/Earning_call/Final_labled_data.csv")
data = pd.DataFrame(data)

#split the data
train_data = data[['transcript' , 'sentiment']]
train_data=data[:10]
train_data.head(20)

,Unnamed: 0,transcript,summary,sentiment,stock_price
0,0,"Good day, and welcome to the Bilibili 2020 Sec...",Bilibili 2020 Second Quarter Earnings Confere...,positive,increase
1,1,Thank you for standing by. This is the confere...,Griffon Corporation 's Third Quarter 2020 Ear...,negative,decrease
2,2,Good day and welcome to the September 2019 Qua...,"Tim Archer, President Chief Executive Officer...",negative,decrease
3,3,"Good day, everyone and thank you for participa...",BBSI 's financial results Third Quarter Ended...,negative,decrease
4,4,Greetings and welcome to the Caesarstone Limit...,Caesarstone Limited Second Quarter 2019 Earni...,negative,decrease
5,5,"Good afternoon, and welcome to the Green Dot T...",Comments may include forward-looking statemen...,negative,decrease
6,6,"Ladies and gentlemen, thank you for standing b...","KAR Auction Services, Inc Q2 2020 Earnings Co...",negative,decrease
7,7,"Ladies and gentlemen, thank you for standing b...",Enerpac Tool Group 's Second Quarter Earnings...,negative,decrease
8,8,Greetings and welcome to the ABM Industries' S...,ABM Industries ' Second Quarter 2021 Earnings...,positive,increase
9,9,Good morning and welcome to Super Group's seco...,Super Group 's second quarter 2022 earnings c...,negative,decrease


In [ ]:
SA_data = train_data[['transcript' , 'sentiment']].copy()

SA_data['sentiment_encoded'] = train_data['sentiment'].apply(lambda x: 1 if x == 'positive' else 0)
middle_start = 2500
middle_end = 3012
SA_data['transcript_middle'] = train_data['transcript'].apply(lambda x: x[middle_start:middle_end])


X = SA_data.transcript_middle.values
y = SA_data.sentiment_encoded.values

#split
X_train, X_val, y_train, y_val =train_test_split(X, y, test_size=0.2, random_state=2020)
SA_data.head()

,transcript,sentiment,sentiment_encoded,transcript_middle
0,"Good day, and welcome to the Bilibili 2020 Sec...",positive,1,"2 million, up 55% and DAUs were up 52% to 51 m..."
1,Thank you for standing by. This is the confere...,negative,0,"vot out of the capital-intensive, commodity-dr..."
2,Good day and welcome to the September 2019 Qua...,negative,0,ghout the world for their contributions to the...
3,"Good day, everyone and thank you for participa...",negative,0,% which was lower than Q3 of '18 and lower tha...
4,Greetings and welcome to the Caesarstone Limit...,negative,0,"novation, production and supply chain to our b..."


In [ ]:
len(X_train)

8

In [ ]:
########################################################################################
#  Load test data
########################################################################################
tst_data = pd.read_csv("/content/gdrive/MyDrive/3owais_Capital/Earning_call/Final_labled_data.csv")

test_data=tst_data.copy()
test_data=test_data[['transcript']]
test_data['transcript_middle'] = data['transcript'].apply(lambda x: x[middle_start:middle_end])
test_data=test_data[['transcript_middle']]
test_data.head()


,transcript_middle
0,"2 million, up 55% and DAUs were up 52% to 51 m..."
1,"vot out of the capital-intensive, commodity-dr..."
2,ghout the world for their contributions to the...
3,% which was lower than Q3 of '18 and lower tha...
4,"novation, production and supply chain to our b..."


In [ ]:
########################################################################################
# Load the BERT tokenizer
########################################################################################

SA_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

def preprocessing_for_bert(data):
    # Create empty lists to store outputs
    input_ids = []
    attention_masks = []

    # For every sentence...
    for transcript in data:

        encoded_transcript = SA_tokenizer.encode_plus(
            text=transcript,
            add_special_tokens=True,        # Add `[CLS]` and `[SEP]`
            max_length=MAX_LEN,              # Max length to truncate/pad
            pad_to_max_length=True,         # Pad transcript to max length
            return_attention_mask=True      # Return attention mask
            )

        # Add the outputs to the lists
        input_ids.append(encoded_transcript.get('input_ids'))
        attention_masks.append(encoded_transcript.get('attention_mask'))

    # Convert lists to tensors
    input_ids = torch.tensor(input_ids)
    attention_masks = torch.tensor(attention_masks)

    return input_ids, attention_masks

In [ ]:
########################################################################################
#  Create PyTorch DataLoader
# ########################################################################################

# We will create an iterator for our dataset
# using the torch DataLoader class .This will help save on memory during training
# and boost the training speed.

MAX_LEN = 512
batch_size = 32

# tokenize the train set and the validation set to
train_inputs, train_masks = preprocessing_for_bert(X_train)
val_inputs, val_masks = preprocessing_for_bert(X_val)

# Convert other data types to torch.Tensor
train_labels = torch.tensor(y_train)
val_labels = torch.tensor(y_val)

#create dataloader for the training set
train_data = TensorDataset(train_inputs , train_masks ,train_labels)
train_dataloader = DataLoader(train_data , batch_size = batch_size)

#create a dataloader for the test set
val_data = TensorDataset(val_inputs, val_masks , val_labels)
val_dataloader = DataLoader(val_data , batch_size = batch_size)





# Run `preprocessing_for_bert` on the test set
print('Tokenizing data...')
test_inputs, test_masks = preprocessing_for_bert(test_data.transcript_middle)

# Create the DataLoader for our test set
test_dataset = TensorDataset(test_inputs, test_masks)
test_dataloader = DataLoader(test_dataset, batch_size=32)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Tokenizing data...


In [ ]:
########################################################################################
# Train Our Model
########################################################################################

'''
The usual way to do a classification tasks is with a class called
BertForSequenceClassification, but we want more control. So, we're making our own class called BertClassifier.

In our BertClassifier class, we'll use BERT to understand the text, and then we'll add our own classifier to decide the text class !!
We'll take the special [CLS] token's final result from BERT, and use it to decide the category using our classifier

the [CLS] token serves as a representation of the entire input sequence, it encapsulates the information needed to make decisions about the input text's class.
 '''

class BertClassifier(nn.Module):

    def __init__(self):

        #freez Set to `False` to fine-tune the BERT model as we want

        # Specify hidden size of BERT , hidden size of our classifier, and number of labels
        #hidden_size is the dimensionality of the model's hidden layers (768 for bert-base-uncased )
        D_in, H, D_out = 768, 50, 2

        #instantiate a BertModel object
        self.bert = BertModel.from_pretrained('bert_base_uncased')

        #instantiate an one-layer feed forward classifier
        self.classifier= nn.Sequential(
            nn.Linear(D_in, H),
            nn.ReLU(),
            #nn.Dropout(0.5),
            nn.Linear(H, D_out)
        )

        # Freeze the BERT model
        if freeze_bert:
            for param in self.bert.parameters():
                param.requires_grad = False


    def forward(self, input_ids, attention_mask):
        """
        Feed input to BERT and the classifier to compute logits.
        """
        # Feed input to BERT
        outputs = self.bert(input_ids=input_ids,
                            attention_mask=attention_mask)

        # Extract the last hidden state of the token `[CLS]` for classification task
        last_hidden_state_cls = outputs[0][:, 0, :]

        # Feed input to classifier to compute logits
        logits = self.classifier(last_hidden_state_cls)

        return logits


In [ ]:
########################################################################################
# Optimizer & Learning Rate Scheduler
########################################################################################

'''
To fine-tune our Bert Classifier, we need to create an optimizer.
The authors recommend following hyper-parameters:
Batch size: 16 or 32
Learning rate (Adam): 5e-5, 3e-5 or 2e-5
Number of epochs: 2, 3, 4
'''
from transformers import AdamW, get_linear_schedule_with_warmup
def initialize_model(epochs=4):
    """Initialize the Bert Classifier, the optimizer and the learning rate scheduler.
    """
    # Instantiate Bert Classifier
    bert_classifier = BertClassifier()

    # Tell PyTorch to run the model on GPU
    bert_classifier.to(device)

    # Create the optimizer
    optimizer = AdamW(bert_classifier.parameters(),
                      lr=5e-5,    # Default learning rate
                      eps=1e-8    # Default epsilon value
                      )

    # Total number of training steps
    total_steps = len(train_dataloader) * epochs

    # Set up the learning rate scheduler
    scheduler = get_linear_schedule_with_warmup(optimizer,
                                                num_warmup_steps=0, # Default value
                                                num_training_steps=total_steps)
    return bert_classifier, optimizer, scheduler


In [ ]:
########################################################################################
# Training Loop
########################################################################################

import random
import time

# Specify loss function
loss_fn = nn.CrossEntropyLoss()

# def set_seed(seed_value=42):
#     """Set seed for reproducibility.
#     """
#     random.seed(seed_value)
#     np.random.seed(seed_value)
#     torch.manual_seed(seed_value)
#     torch.cuda.manual_seed_all(seed_value)

def train(model, train_dataloader, val_dataloader=None, epochs=4, evaluation=False):
    """Train the BertClassifier model.
    """
    # Start training loop
    print("Start training...\n")
    for epoch_i in range(epochs):
        # =======================================
        #               Training
        # =======================================
        # Print the header of the result table
        print(f"{'Epoch':^7} | {'Batch':^7} | {'Train Loss':^12} | {'Val Loss':^10} | {'Val Acc':^9} | {'Elapsed':^9}")
        print("-"*70)

        # Measure the elapsed time of each epoch
        t0_epoch, t0_batch = time.time(), time.time()

        # Reset tracking variables at the beginning of each epoch
        total_loss, batch_loss, batch_counts = 0, 0, 0

        # Put the model into the training mode
        model.train()

        # For each batch of training data...
        # for step, batch in enumerate(train_dataloader):
        for step, batch in tqdm(enumerate(train_dataloader), total=len(train_dataloader), desc=f"Epoch {epoch_i + 1} Training"):
            batch_counts +=1
            # Load batch to GPU
            b_input_ids, b_attn_mask, b_labels = tuple(t.to(device) for t in batch)

            # Zero out any previously calculated gradients
            model.zero_grad()

            # Perform a forward pass. This will return logits.
            logits = model(b_input_ids, b_attn_mask)

            # Compute loss and accumulate the loss values
            loss = loss_fn(logits, b_labels)
            batch_loss += loss.item()
            total_loss += loss.item()

            # Perform a backward pass to calculate gradients
            loss.backward()

            # Clip the norm of the gradients to 1.0 to prevent "exploding gradients"
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            # Update parameters and the learning rate
            optimizer.step()
            scheduler.step()

            # Print the loss values and time elapsed for every 20 batches
            if (step % 20 == 0 and step != 0) or (step == len(train_dataloader) - 1):
                # Calculate time elapsed for 20 batches
                time_elapsed = time.time() - t0_batch

                # Print training results
                print(f"{epoch_i + 1:^7} | {step:^7} | {batch_loss / batch_counts:^12.6f} | {'-':^10} | {'-':^9} | {time_elapsed:^9.2f}")

                # Reset batch tracking variables
                batch_loss, batch_counts = 0, 0
                t0_batch = time.time()

        # Calculate the average loss over the entire training data
        avg_train_loss = total_loss / len(train_dataloader)

        print("-"*70)
        # =======================================
        #               Evaluation
        # =======================================
        if evaluation == True:
            # After the completion of each training epoch, measure the model's performance
            # on our validation set.
            val_loss, val_accuracy = evaluate(model, val_dataloader)

            # Print performance over the entire training data
            time_elapsed = time.time() - t0_epoch

            print(f"{epoch_i + 1:^7} | {'-':^7} | {avg_train_loss:^12.6f} | {val_loss:^10.6f} | {val_accuracy:^9.2f} | {time_elapsed:^9.2f}")
            print("-"*70)
        print("\n")

    print("Training complete!")


def evaluate(model, val_dataloader):
    """After the completion of each training epoch, measure the model's performance
    on our validation set.
    """
    # Put the model into the evaluation mode. The dropout layers are disabled during
    # the test time.
    model.eval()

    # Tracking variables
    val_accuracy = []
    val_loss = []

    # For each batch in our validation set...
    # for batch in val_dataloader:
    for batch in tqdm(val_dataloader, total=len(val_dataloader), desc=f"Epoch {epoch_i + 1} Evaluation"):

        # Load batch to GPU
        b_input_ids, b_attn_mask, b_labels = tuple(t.to(device) for t in batch)

        # Compute logits
        with torch.no_grad():
            logits = model(b_input_ids, b_attn_mask)

        # Compute loss
        loss = loss_fn(logits, b_labels)
        val_loss.append(loss.item())

        # Get the predictions
        preds = torch.argmax(logits, dim=1).flatten()

        # Calculate the accuracy rate
        accuracy = (preds == b_labels).cpu().numpy().mean() * 100
        val_accuracy.append(accuracy)

    # Compute the average accuracy and loss over the validation set.
    val_loss = np.mean(val_loss)
    val_accuracy = np.mean(val_accuracy)

    return val_loss, val_accuracy


In [ ]:
########################################################################################
# Let's train
########################################################################################

bert_classifier, optimizer, scheduler = initialize_model(epochs=2)
train(bert_classifier, train_dataloader, val_dataloader, epochs=2, evaluation=True)

OSError: ignored

In [ ]:
########################################################################################
#  Evaluation on Validation Set
########################################################################################
import torch.nn.functional as F

def bert_predict(model, test_dataloader):
    """Perform a forward pass on the trained BERT model to predict probabilities
    on the test set.
    """
    model.eval()

    all_logits = []

    # For each batch in our test set...
    for batch in test_dataloader:
        # Load batch to GPU
        b_input_ids, b_attn_mask = tuple(t.to(device) for t in batch)[:2]

        # Compute logits
        with torch.no_grad():
            logits = model(b_input_ids, b_attn_mask)
        all_logits.append(logits)

    # Concatenate logits from each batch
    all_logits = torch.cat(all_logits, dim=0)

    # Apply softmax to calculate probabilities
    probs = F.softmax(all_logits, dim=1).cpu().numpy()

    return probs